# CYPLAN255
### Urban Informatics and Visualization

# Lecture 10 -- Intro to Data Visualization
*******
February 26, 2024
<img title="Aerial view of the Japanese aircraft carrier Sōryū evading an air attack during the Battle of Midway, 1942" src="https://i.redd.it/xmh9uljdl0ca1.jpg" width=50% align="right">

# Agenda
1. Announcements
2. Review of last session
3. Data Viz
4. For next time
5. Questions


# 1. Announcements

- Assignment 3 Due Weds
- Meiqing to lead GitHub Pages tutorial
   - _will be recorded_

# 2. Review

## 2.1 Imports and load data

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

In [ ]:
sf1 = pd.read_hdf('data/bay_sf1_small.h5', 'sf1_extract')
sf1.head()

In [ ]:
rentals = pd.read_csv(
    'data/sfbay_geocoded.csv',
    usecols=['rent', 'bedrooms', 'sqft', 'fips_block', 'longitude', 'latitude'],
    dtype={'fips_block': str}  # load fips_block as str, numeric type will drop leading zero
)  
rentals.head()

## 2.2 Cleaning and transforming

In [ ]:
county_fips_to_name = {
    '001': 'Alameda', '013': 'Contra Costa', '041': 'Marin', '055': 'Napa', '075': 'San Francisco',
    '081': 'San Mateo', '085': 'Santa Clara', '095': 'Solano', '097': 'Sonoma'}

In [ ]:
sf1.insert(4, 'county_name', sf1['county'].replace(county_fips_to_name))

In [ ]:
sf1['pct_rent'] = sf1['H0040004'] / sf1['H0040001'] * 100
sf1['pct_black'] = sf1['P0030003'] / sf1['P0030001'] * 100
sf1['pct_asian'] = sf1['P0030005'] / sf1['P0030001'] * 100
sf1['pct_white'] = sf1['P0030002'] / sf1['P0030001'] * 100
sf1['pct_hisp'] = sf1['P0040003'] / sf1['P0040001'] * 100
sf1['pct_vacant'] = sf1['H0050001'] / sf1['H00010001'] * 100
sf1['pop_sqmi'] = (sf1['P0010001'] / (sf1['arealand'] / 2589988))
sf1 = sf1[sf1['P0030001'] > 0]  # drop unpopulated census blocks

## 2.3 Merging data tables

In [ ]:
rentals_sf1 = pd.merge(rentals, sf1, left_on='fips_block', right_on='blockfips')
rentals_sf1.head()

## 2.4 Plotting with pandas

Let's visualize average rent by `county` and `bedrooms`. We can use a multi-column group-by for this:

In [ ]:
rentals_sf1.groupby(['county_name', 'bedrooms'])['rent'].mean().plot(kind='bar')

Note that in one line of pandas code we can aggregate, transform, and plot our data. But this plot has some issues. Let's see if we can improve it.

In [ ]:
_ = rentals_sf1.groupby(['county_name', 'bedrooms'])['rent'].mean().unstack().plot(
    kind='bar', figsize=(14,6), title='Avg. Rents by County and # Bedrooms', ylabel='rent ($)', xlabel='county')

If we re-order the sequence of our group-by columns, we'll swap the rows and columns of the dataframe returned by `unstack()`:

In [ ]:
rentals_sf1.groupby(['bedrooms', 'county_name'])['rent'].mean().unstack().plot(
    kind='bar', figsize=(14,6), title='Average Rents by County and # Bedrooms',
    ylabel='rent ($)', xlabel='# bedrooms')

## 2.5 Pivot tables and cross-tabulations

For all you Excel wizards out there thinking "this is dumb it's just a pivot table"...you're right!

In [ ]:
pd.pivot_table(rentals_sf1, values='rent', index=['bedrooms'], columns=['county_name']).plot(
    kind='bar', figsize=(14,6), title='Average Rents by County and Bedrooms')

Behind the scenes, pivot table is just doing a group-by and taking the mean.

In [ ]:
pd.pivot_table(rentals_sf1, values='rent', index='bedrooms', columns='county_name')

But you could specify other aggregation functions if you wanted. You can also add other useful features like marginals.

In [ ]:
pd.pivot_table(rentals_sf1, values='rent', index='bedrooms', columns='county_name', aggfunc='count', margins=True)

When you specify `count` as your aggregation method, you are doing **cross-
tabulation**.        

Cross-tabulation is such a common operation that pandas has a built-in function for _it_, too!

In [ ]:
pd.crosstab(rentals_sf1['bedrooms'], rentals_sf1['county_name'], margins=True)

Getting a little fancy here, but cross-tab data lends itself nicely to higher-dimensional data visualization styles like the "heat map" below:

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
_ = sns.heatmap(
    pd.crosstab(rentals_sf1['county_name'], rentals_sf1['bedrooms'], normalize='columns'), square=True, ax=ax)

# 3. Intro to Data Visualization

- 3.1. Ethics of Data Visualization 
- 3.2. Conceptual Overview
- 3.3. Matplotlib
- 3.4. Seaborn

## 3.1 Ethical Data Visualization

It is exceedingly easy to lie with data visualization. You **will** be tempted to do so, if only subconsciouly, in order to nudge your results towards support for your hypotheses. Be aware, and learn to recognize this tendency in yourself.

A victimless crime? Perhaps.

But perhaps not:

In [ ]:
%%html
<iframe src="https://web.archive.org/web/20211112185644/https://www.cbsnews.com/news/how-purdue-used-misleading-charts-to-hide-oxycontins-addictive-power/" width=100% height=800></iframe>

## 3.2 Conceptual overview

### 3.2.1 Categorical vs. Continuous data

In [ ]:
fig, axarr = plt.subplots(1, 2, figsize=(13,5))
rentals_sf1['county_name'].value_counts(sort=False).plot(kind='bar', ax=axarr[0], title='County Name')
axarr[0].set_ylabel("count")
rentals_sf1['rent'].plot(kind='kde', ax=axarr[1], title='Price ($)')

![](https://matplotlib.org/stable/_images/sphx_glr_categorical_variables_001_2_00x.png)


### 3.2.2 Building blocks

- **shape**: points, lines, bars, areas (fill/patch)
- **size**: of shape, figure, etc.
- **color**: hue, opacity, gradient (colormap)
- **axes (dimensionality)**: tick marks, labels
- **auxiliary data**: legend, margin plots, annotation

### 3.2.3 Dimensionality

**Dimensionality**: number of axes of variation

### Question 1
How many dimensions are shown in the following examples?

<img src="https://assets.bwbx.io/images/users/iqjWHBFdfxIU/iCIEic94rsFk/v0/-1x-1.png" width=800px >

![](https://upload.wikimedia.org/wikipedia/commons/thumb/8/80/SoilTexture_USDA.png/500px-SoilTexture_USDA.png)

<img src="https://raw.githubusercontent.com/zonination/perceptions/master/joy1.png" width=700px>

<img src="https://flowingdata.com/wp-content/uploads/2013/09/most-unisex-names2.png" width=600px>

In [ ]:
%%html
<iframe src="//www.gapminder.org/tools/?embedded=true#$ui;&chart-type=bubbles&url=v1" style="width: 100%; height: 500px; margin: 0 0 0 0; border: 1px solid grey;" allowfullscreen></iframe>

**EXAMPLES:**

- **1d**: [histogram](https://matplotlib.org/stable/plot_types/stats/hist_plot.html), [kernel density estimate (KDE)](https://seaborn.pydata.org/tutorial/distributions.html#kernel-density-estimation), [rugplot](https://seaborn.pydata.org/generated/seaborn.rugplot.html#seaborn.rugplot)/[eventplot](https://matplotlib.org/stable/plot_types/stats/eventplot.html), [pie chart](https://matplotlib.org/stable/plot_types/stats/pie.html)

- **1.5d**: [boxplot](https://matplotlib.org/stable/gallery/pyplots/boxplot_demo_pyplot.html), [grouped histogram](https://seaborn.pydata.org/tutorial/distributions.html#conditioning-on-other-variables), [grouped KDE](https://seaborn.pydata.org/tutorial/distributions.html#id1)

- **2d**: [scatter](https://matplotlib.org/stable/plot_types/basic/scatter_plot.html), [line](https://matplotlib.org/stable/plot_types/basic/plot.html), [bar](https://matplotlib.org/stable/plot_types/basic/bar.html#sphx-glr-plot-types-basic-bar-py), [boxplot](https://matplotlib.org/stable/gallery/pyplots/boxplot_demo_pyplot.html), maps (?)

- **2.5d**: [categorical scatter](https://seaborn.pydata.org/tutorial/categorical.html#categorical-scatterplots), [grouped boxplot](https://seaborn.pydata.org/tutorial/categorical.html#boxplots), [grouped line](https://seaborn.pydata.org/generated/seaborn.lineplot.html), [grouped scatter](https://seaborn.pydata.org/tutorial/relational.html#relating-variables-with-scatter-plots), [bubble](https://plotly.com/python/bubble-charts/)

- **3d+**: [heatmap](https://seaborn.pydata.org/generated/seaborn.heatmap.html), [contour](https://matplotlib.org/stable/gallery/images_contours_and_fields/contour_demo.html), [bivariate distribution](https://seaborn.pydata.org/tutorial/distributions.html#visualizing-bivariate-distributions), [hexbin](https://matplotlib.org/stable/plot_types/stats/hexbin.html), [raster images](https://matplotlib.org/stable/plot_types/arrays/imshow.html), 


## 3.3. Matplotlib

Matplotlib is to data viz what NumPy is to data analysis in Python. Nearly every other plotting library is built on top of it. For this reason, Matplotlib is the most flexible and powerful of Python's data viz packages. Almost everything that can be done in the more sophisticated plotting libraries can be implemented in pure Matplotlib with a little bit of extra work.



In practice, I find Matplotlib most useful for two scenarios:
1. quick 1- or 2d data viz (histograms, scatterplots, etc.) for array-based data
2. control of figure components for non-standard plot types

### 3.3.1. The basics: `Figure` vs. `Axes` vs. the rest 

<img src="https://matplotlib.org/stable/_images/anatomy.png" width=700px>

In [ ]:
plt.hist(rentals_sf1['P0010001'])
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.hist(rentals_sf1['P0010001'])
plt.show()

In [ ]:
fig, ax = plt.subplots()
_ = ax.hist(rentals_sf1['P0010001'])

### 3.3.2. Subplots and axes

### Question 2
Can you spot the difference between the next two plots?

In [ ]:
fig = plt.figure(1, figsize=(10,8), )
fig.suptitle("2010 Racial and Ethnic Distribution by Census Block, San Francisco Bay Area", fontsize=16)

ax1 = plt.subplot(221)
ax1.hist(rentals_sf1['pct_asian'], bins=50)
ax1.set_title("Asian")
ax1.set_xlabel('% Population in Block')
ax1.set_ylabel('Total Census Blocks')

ax2 = plt.subplot(222)
ax2.hist(rentals_sf1['pct_black'], bins=50)
ax2.set_title("Black")
ax2.set_xlabel('% Population in Block')
ax2.set_ylabel('Total Census Blocks')

ax3 = plt.subplot(223)
ax3.hist(rentals_sf1['pct_hisp'], bins=50)
ax3.set_title("Hispanic")
ax3.set_xlabel('% Population in Block')
ax3.set_ylabel('Total Census Blocks')

ax4 = plt.subplot(224)
ax4.hist(rentals_sf1['pct_white'], bins=50)
ax4.set_title("White")
ax4.set_xlabel('% Population in Block')
ax4.set_ylabel('Total Census Blocks')

plt.subplots_adjust(wspace=.25, hspace=.25)
plt.show()

In [ ]:
fig, axarr = plt.subplots(2, 2, figsize=(10,8), sharey=True, sharex=True)
fig.suptitle("2010 Racial and Ethnic Distribution by Census Block, San Francisco Bay Area", fontsize=16)

axarr[0, 0].hist(rentals_sf1['pct_asian'], bins=50)
axarr[0, 0].set_title("Asian")
axarr[0, 0].set_xlabel('% Population in Block')
axarr[0, 0].set_ylabel('Total Census Blocks')

axarr[0, 1].hist(rentals_sf1['pct_black'], bins=50)
axarr[0, 1].set_title("Black")
axarr[0, 1].set_xlabel('% Population in Block')
axarr[0, 1].set_ylabel('Total Census Blocks')

axarr[1, 0].hist(rentals_sf1['pct_hisp'], bins=50)
axarr[1, 0].set_title("Hispanic")
axarr[1, 0].set_xlabel('% Population in Block')
axarr[1, 0].set_ylabel('Total Census Blocks')

axarr[1, 1].hist(rentals_sf1['pct_white'], bins=50)
axarr[1, 1].set_title("White")
axarr[1, 1].set_xlabel('% Population in Block')
axarr[1, 1].set_ylabel('Total Census Blocks')

plt.subplots_adjust(wspace=.25, hspace=.25)
plt.show()

Let's clean up this code a bit

In [ ]:
import matplotlib.ticker as ticker

fig, axarr = plt.subplots(2,2, figsize=(10,8), sharey=True, sharex=True)
fig.suptitle("2010 Racial and Ethnic Distribution by Census Block, San Francisco Bay Area", fontsize=16)

cats = ['asian', 'black', 'hisp', 'white']

for i, ax in enumerate(axarr.flatten()):
    ax.hist(rentals_sf1['pct_{0}'.format(cats[i])], bins=50)
    ax.set_title(cats[i])
    ax.set_xlabel('% Population in Block')
    ax.set_ylabel('Total Census Blocks')
    ax.xaxis.set_major_formatter(ticker.PercentFormatter(xmax=100))

plt.subplots_adjust(wspace=.25, hspace=.25)
plt.show()

In [ ]:
fig, ax= plt.subplots(figsize=(10, 4))
fig.suptitle("2010 Racial and Ethnic Distribution by Census Block, San Francisco Bay Area", fontsize=16)

cats = ['asian', 'black', 'hisp', 'white']

for cat in cats:
    ax.hist(rentals_sf1['pct_{0}'.format(cat)], bins=50, alpha=0.3, label=cat)
    ax.set_xlabel('% Population in Block')
    ax.set_ylabel('Total Census Blocks')
    ax.xaxis.set_major_formatter(ticker.PercentFormatter(xmax=100))
    
ax.legend()

plt.subplots_adjust(wspace=.25, hspace=.25)
plt.show()

### 3.3.3 Bivariate data

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
foo = ax.scatter(rentals_sf1['sqft'], rentals_sf1['rent'], c='g', alpha=0.2, s=0.4)
ax.set_xlabel("Size (ft$^2$)", fontsize=15)
_ = ax.set_ylabel('Rent ($)', fontsize=15)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.set_title("SF Bay Area Craigslist Rental Listings", fontsize=20)
ax.set_xlabel('Bedrooms', fontsize=15)
ax.set_ylabel('Rent ($)', fontsize=15)
foo = ax.scatter(
    rentals_sf1['bedrooms'], rentals_sf1['rent'], c=np.log(rentals_sf1['sqft']), alpha=.1, s=50)
cbar = fig.colorbar(foo)
cbar.set_label("log(size) (ft$^2$)", fontsize=15)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.set_title("SF Bay Area Craigslist Rental Listings", fontsize=20)
ax.set_xlabel('Bedrooms', fontsize=15)
ax.set_ylabel('Rent ($)', fontsize=15)
_ = ax.boxplot([x['rent'] for i, x in rentals_sf1.groupby('bedrooms')])

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sns.boxplot(data=rentals_sf1, x='bedrooms', y='rent', ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sns.stripplot(data=rentals_sf1, x='bedrooms', y='rent', alpha=0.1, ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(13, 6))
sns.violinplot(data=rentals_sf1, x='bedrooms', y='rent',ax=ax)

## 3.4 Seaborn

### 3.4.1 The basics

Matplotlib, for DataFrames! In fact, it works directly on top of `fig` and `ax` objects:

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sns.stripplot(data=rentals_sf1, x='bedrooms', y='rent', alpha=0.1, ax=ax)
ax.set_title("SF Bay Area Craigslist Rental Listings", fontsize=20)
ax.set_xlabel('Bedrooms', fontsize=15)
_ = ax.set_ylabel('Rent ($)', fontsize=15)
fig.set_facecolor('lightgrey')

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
sns.kdeplot(data=rentals_sf1, x="rent", hue='county_name', fill=True, ax=ax)

### 3.4.2 Facets

Seaborn makes it easy to create a grid of subplots on-the-fly. In Seaborn parlance, each subplot is a `Facet`, and the grid is a `FacetGrid`. Seaborn allows you to specify a `FacetGrid` in which the rows/columns correspond to column values from your dataframe. Below, we define a FacetGrid where the facets are defined by `county_name`, and use the `col_wrap` argument to tell Seaborn to start a new row after every 3rd column.

In [ ]:
g = sns.FacetGrid(rentals_sf1, col="county_name", col_wrap=3, height=2.5, aspect=1.5)
g.map(sns.histplot, "rent")

In [ ]:
g = sns.FacetGrid(rentals_sf1, col="county_name", col_wrap=3, height=2.5, aspect=1.5)
g.map(sns.kdeplot, "rent", fill=True)

In [ ]:
from matplotlib import rcParams

with sns.axes_style({"style": "white", "axes.facecolor": (0, 0, 0, 0)}):
    g = sns.FacetGrid(rentals_sf1, row="county_name", hue="county_name", aspect=20, height=.75)
    g.map(sns.kdeplot, "rent",
          bw_adjust=.5, clip_on=False,
          fill=True, alpha=.5, linewidth=1.5)
    g.figure.subplots_adjust(hspace=-.5)
    g.set_titles("")
    g.set(yticks=[])
    g.despine( left=True)
    g.set_axis_labels("Rent ($)", "", fontsize=15)
    g.set_titles(row_template='{row_name}')
    for ax in g.axes:
        title = ax[0].get_title()
        ax[0].set_title("")
        ax[0].set_ylabel(title, rotation=0, loc='bottom', labelpad=0)


### 3.4.3 High-dimension plots

In [ ]:
%time
fig, ax = plt.subplots(figsize=(10,7))
sns.kdeplot(
    x="sqft", y="rent", data=rentals_sf1, thresh=0, fill=True, cmap='magma', alpha=0.7)

In [ ]:
%time
sns.jointplot(
    x="sqft", y="rent", kind="hex", data=rentals_sf1, cmap='magma', joint_kws={"mincnt": 1, "gridsize":30})

In [ ]:
sns.pairplot(rentals_sf1[['rent', 'sqft', 'pop_sqmi', 'pct_white']],
    plot_kws = {'alpha': 0.05, 's': 1, 'edgecolor': 'none', 'facecolor':'k'});

### 3.4.4. A bit of color theory

In [ ]:
%%html
<iframe src="https://ai.googleblog.com/2019/08/turbo-improved-rainbow-colormap-for.html" width=100% height=700></iframe>

#### 3.4.4.1 The Three Components of Color

1. Hue

In [ ]:
sns.color_palette("hls", 8)

2. Saturation

In [ ]:
sns.color_palette("blend:grey,b", 8)

3. Lightness

In [ ]:
sns.cubehelix_palette(8, start=.25, rot=-.25, reverse=True)

#### 3.4.4.2. Color Palettes

Categorical/Qualitative

<center><img src="https://seaborn.pydata.org/_images/color_palettes_22_0.svg" width=500></center>

In [ ]:
sns.color_palette("dark", 10)

In [ ]:
# colorbrewer palettes
sns.color_palette("Set2", 7)

Circular color systems

In [ ]:
# HLS
sns.color_palette("hls", 7)

In [ ]:
# HUSL
sns.color_palette("husl", 7)

Sequential

In [ ]:
# perceptually uniform
sns.color_palette("rocket_r", 10)

In [ ]:
# cubehelix
sns.color_palette("ch:start=.5,rot=-.5", 10)

Divergent

In [ ]:
sns.color_palette("Spectral", 10)

In [ ]:
sns.color_palette("coolwarm", 10)

In [ ]:
# perceptually uniform
sns.color_palette("vlag", 10)

In [ ]:
# custom
sns.diverging_palette(250, 30, s=99, l=65, center="dark", n=10)

Matplotlib colormaps

In [ ]:
cmaps = [('Perceptually Uniform Sequential', [
            'viridis', 'plasma', 'inferno', 'magma']),
         ('Sequential', [
            'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
            'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
            'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn']),
         ('Sequential (2)', [
            'binary', 'gist_yarg', 'gist_gray', 'gray', 'bone', 'pink',
            'spring', 'summer', 'autumn', 'winter', 'cool', 'Wistia',
            'hot', 'afmhot', 'gist_heat', 'copper']),
         ('Diverging', [
            'PiYG', 'PRGn', 'BrBG', 'PuOr', 'RdGy', 'RdBu',
            'RdYlBu', 'RdYlGn', 'Spectral', 'coolwarm', 'bwr', 'seismic']),
         ('Qualitative', [
            'Pastel1', 'Pastel2', 'Paired', 'Accent',
            'Dark2', 'Set1', 'Set2', 'Set3',
            'tab10', 'tab20', 'tab20b', 'tab20c']),
         ('Miscellaneous', [
            'flag', 'prism', 'ocean', 'gist_earth', 'terrain', 'gist_stern',
            'gnuplot', 'gnuplot2', 'CMRmap', 'cubehelix', 'brg', 'hsv',
            'gist_rainbow', 'rainbow', 'jet',  'turbo', 'nipy_spectral', 'gist_ncar'])]

nrows = [len(cmap_list) for cmap_category, cmap_list in cmaps]
gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))


def plot_color_gradients(cmap_category, cmap_list, nrows):
    fig, axes = plt.subplots(nrows=nrows, figsize=(10, nrows / 3))
    fig.subplots_adjust(top=0.95, bottom=0.01, left=0.2, right=0.99)
    axes[0].set_title(cmap_category + ' colormaps', fontsize=14)

    for ax, name in zip(axes, cmap_list):
        ax.imshow(gradient, aspect='auto', cmap=plt.get_cmap(name))
        pos = list(ax.get_position().bounds)
        x_text = pos[0] - 0.01
        y_text = pos[1] + pos[3]/2.
        fig.text(x_text, y_text, name, va='center', ha='right', fontsize=10)

    # Turn off *all* ticks & spines, not just the ones with colormaps.
    for ax in axes:
        ax.set_axis_off()

In [ ]:
for i, cmap in enumerate(cmaps):
    cmap_category, cmap_list = cmap
    plot_color_gradients(cmap_category, cmap_list, nrows[i])

#### 3.4.4.3 More on Color
- https://www.boronine.com/2012/03/26/Color-Spaces-for-Human-Beings/
- https://colorbrewer2.org/
- https://earthobservatory.nasa.gov/blogs/elegantfigures/2013/08/05/subtleties-of-color-part-1-of-6/
- https://matplotlib.org/2.0.2/users/colormaps.html

# 4. For next time

- Data viz. examples
- HW3

# 5. Questions?